In [31]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from deel.lip.activations import GroupSort2
from deel.lip.layers import (
    FrobeniusDense,
    ScaledL2NormPooling2D,
    SpectralConv2D,
    SpectralDense,
)
from deel.lip.losses import MulticlassHKR, MulticlassKR
from deel.lip.model import Sequential, Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Flatten, Input, Activation,Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [32]:
layers = [
    SpectralDense(100, kernel_initializer="orthogonal"),
    Activation("relu"),
    #GroupSort2(),
    SpectralDense(100, kernel_initializer="orthogonal"),
    Activation("relu"),
    #GroupSort2(),
    SpectralDense(32, kernel_initializer="orthogonal"),
    Activation("relu"),
    #GroupSort2(),
    FrobeniusDense(10, activation=None, use_bias=False, kernel_initializer="orthogonal"),
]

model = Sequential(
    layers,
    k_coef_lip=1.0,
    name="hkr_model",
)

In [33]:
model.compile(
    # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
    # note also in the case of lipschitz networks, more robustness require more parameters.
    loss=MulticlassHKR(alpha=50, min_margin=0.05),
    optimizer=Adam(1e-3),
    metrics=["accuracy", MulticlassKR()],
)


# load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# standardize and reshape the data
x_train = np.expand_dims(x_train, -1)
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = np.expand_dims(x_test, -1)
x_test = (x_test - mean) / std
# one hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

In [34]:
import deel

In [35]:
from tensorflow.keras.models import Sequential, Model
isinstance(Sequential(), Model)

True

In [39]:
vanilla_model

In [16]:
isinstance(model.__class__, deel.lip.model.Functional)

AttributeError: module 'deel.lip.model' has no attribute 'Functional'

# fit the model
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=10,
    validation_data=(x_test, y_test),
    shuffle=True,
    verbose=0,
)

In [37]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 1ms/step - loss: 8.1205 - accuracy: 0.1133 - MulticlassKR: 0.0096


[8.120519638061523, 0.11330000311136246, 0.009622250683605671]

In [38]:
# once training is finished you can convert
# SpectralDense layers into Dense layers and SpectralConv2D into Conv2D
# which optimize performance for inference
vanilla_model = model.vanilla_export()

In [6]:
vanilla_model.summary()

Model: "hkr_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spectral_dense (Dense)      (None, 100)               78500     
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 spectral_dense_1 (Dense)    (None, 100)               10100     
                                                                 
 activation_1 (Activation)   (None, 100)               0         
                                                                 
 spectral_dense_2 (Dense)    (None, 32)                3232      
                                                                 
 activation_2 (Activation)   (None, 32)                0         
                                                                 
 frobenius_dense (Dense)     (None, 10)                32

In [7]:
from decomon import get_adv_box
from decomon.models import clone

In [8]:
tmp = vanilla_model.predict(x_test[:1])[0]
tmp.sort()
eps_ = (tmp[-1] - tmp[-2]) / (2)

1/1 [==============================] - 0s 65ms/step


In [9]:
eps_

0.11633595824241638

In [10]:
for i in range(784):
    mask = np.zeros((1, 784))
    mask[0, i]= eps_
    label = np.argmax(vanilla_model.predict(x_test[:1]+mask, verbose=0))
    if label!=7:
        import pdb; pdb.set_trace()


> /var/folders/mv/rks11p6n68n4p1fqvdm9z5_w0000gn/T/ipykernel_18038/3993656677.py(1)<module>()
----> 1 for i in range(784):
      2     mask = np.zeros((1, 784))
      3     mask[0, i]= eps_
      4     label = np.argmax(vanilla_model.predict(x_test[:1]+mask, verbose=0))
      5     if label!=7:

ipdb> q


In [10]:
C = Input((10, 10))
convex_domain = {"name": "ball", "p": 2, "eps": tf.Variable(eps_, dtype='float32')}

In [12]:
decomon_model_adv = clone(vanilla_model, back_bounds=[C], convex_domain=convex_domain, method='forward-hybrid')

> /Users/ducoffe/Documents/Code/EMACS/decomon/src/decomon/models/forward_cloning.py(204)convert_forward_functional_model()
    202                     output = converted_layer(prepare_inputs_for_layer(output))
    203                     if len(converted_layers) > 1:
--> 204                         output_map[f"{id(node)}_{converted_layer.name}"] = wrap_outputs_from_layer_in_list(output)
    205                 layer_map[id(node)] = converted_layers
    206             output_map[id(node)] = wrap_outputs_from_layer_in_list(output)

ipdb> p output
[<KerasTensor: shape=(None, 784) dtype=float32 (created by layer 'spectral_dense_input_decomon_0')>, <KerasTensor: shape=(None, 784) dtype=float32 (created by layer 'spectral_dense_input_decomon_0')>, <KerasTensor: shape=(None, 784, 784) dtype=float32 (created by layer 'spectral_dense_input_decomon_0')>, <KerasTensor: shape=(None, 784) dtype=float32 (created by layer 'spectral_dense_input_decomon_0')>, <KerasTensor: shape=(None, 784) dtype=flo

In [67]:
C_ = np.diag([1.0] * 10)[None] - y_test[:, :, None]
C_i = C_[:1]

In [69]:
decomon_model_adv.predict([x_test[:1], C_i])[0]

1/1 [==============================] - 0s 41ms/step


array([[1939.5995, 1300.9492, 1850.9507, 1886.1985, 1682.9812, 2235.8376,
        2107.2407,    0.    , 2054.0623, 1681.4781]], dtype=float32)

In [57]:
K.set_value(convex_domain['eps'], 1.)

In [ ]:
array([[4466.2153, 5305.5625, 4413.2036, 5390.5273, 4907.2407, 4964.0195,
         5451.3735,    0.    , 5529.8247, 6102.4473]], dtype=float32)

In [ ]:
box = np.concatenate([x_min[:, None], x_max[:, None]], 1)

In [ ]:
upper = decomon_model_adv.predict([box, C_i])

In [ ]:
x_test

In [ ]:
upper

retrive one radis bound

In [ ]:
vanilla_model.predict(x_test[:1]).argmax()

In [ ]:
y_test[:1].argmax()

In [ ]:
tmp = vanilla_model.predict(x_test[:1])[0]
tmp.sort()
eps_ = (tmp[-1] - tmp[-2]) / np.sqrt(2)

In [ ]:
C_i.shape

In [ ]:
decomon_model_adv.predict(x_test[:1])

In [ ]:
decomon_model = clone(vanilla_model, method="crown")